In [1]:
%load_ext autoreload
%autoreload 2

# Load Data

In [2]:
from pathlib import Path
from opensynth.data_modules.lcl_data_module import LCLDataModule
import pytorch_lightning as pl

import matplotlib.pyplot as plt

data_path = Path("../../data/processed/historical/train/lcl_data.csv")
stats_path = Path("../../data/processed/historical/train/mean_std.csv")
outlier_path = Path("../../data/processed/historical/train/outliers.csv")

dm = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=25000, n_samples=50000)
dm.setup()

In [3]:
import torch
from opensynth.models.faraday import FaradayVAE
vae_model = torch.load("vae_model.pt")

/var/folders/lj/tc13qh690297rzt9q9c3qdbm0000gn/T/ipykernel_81511/3246375714.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae_model = torch.load("vae_model.pt")


In [4]:
from opensynth.models.faraday.gaussian_mixture.prepare_gmm_input import encode_data_for_gmm

next_batch = next(iter(dm.train_dataloader()))
input_tensor = encode_data_for_gmm(data=next_batch, vae_module=vae_model)
input_data = input_tensor.detach().numpy()
n_samples = len(input_tensor)

In [5]:
N_COMPONENTS = 25

# Init GMM

In [45]:
from opensynth.models.faraday.new_gmm import gmm_utils

labels_, means_, responsibilities_ = gmm_utils.initialise_centroids(
        X=input_data, n_components=N_COMPONENTS
    )

responsibilities_ = responsibilities_.float()
means_ = means_.float()

In [64]:
import numpy as np
def _estimate_gaussian_covariances_full(resp, X, nk, reg_covar):
    """Estimate the full covariance matrices.

    Parameters
    ----------
    resp : array-like of shape (n_samples, n_components)

    X : array-like of shape (n_samples, n_features)

    nk : array-like of shape (n_components,)

    means : array-like of shape (n_components, n_features)

    reg_covar : float

    Returns
    -------
    covariances : array, shape (n_components, n_features, n_features)
        The covariance matrix of the current components.
    """
    nk = resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps
    means = np.dot(resp.T, X) / nk[:, np.newaxis]

    n_components, n_features = means.shape
    covariances = np.empty((n_components, n_features, n_features))
    for k in range(n_components):
        diff = X - means[k]
        covariances[k] = np.dot(resp[:, k] * diff.T, diff) / nk[k]
        covariances[k].flat[:: n_features + 1] += reg_covar
    return covariances

In [71]:
from opensynth.models.faraday.new_gmm.train_gmm import initialise_gmm_params
from opensynth.models.faraday.new_gmm.new_gmm_model import GaussianMixtureModel

gmm_init_params = initialise_gmm_params(
    X=input_data,
    n_components = N_COMPONENTS
)
torch_gmm = GaussianMixtureModel(
    num_components=N_COMPONENTS,
    num_features = input_data.shape[1],
)
torch_gmm.initialise(gmm_init_params)

In [73]:
torch_log_prob_norm, torch_log_resp = torch_gmm.e_step(input_tensor)

In [85]:
torch_log_resp[0]

tensor([-5.6400e+01, -2.4667e+01, -1.0073e+01, -1.1522e+01, -2.4538e+01,
        -2.6805e+01, -2.9733e+01, -3.1089e+01, -7.4801e+01, -3.2550e+01,
        -2.1851e+01, -1.0096e+01, -2.3091e+01, -4.5575e+01, -5.2834e-04,
        -8.5151e+00, -1.9881e+01, -1.2752e+01, -3.4889e+01, -4.3621e+01,
        -5.7187e+01, -2.8923e+01, -5.3805e+01, -8.9127e+00, -9.2404e+00],
       grad_fn=<SelectBackward0>)

In [84]:
torch_log_prob_norm

tensor(-1.0545, grad_fn=<MeanBackward0>)

In [76]:
torch_prec_chol, torch_weights, torch_means = torch_gmm.m_step(input_tensor, log_reponsibilities=torch_log_resp)

# E-step

In [92]:
### SKLearn:
import numpy as np
from scipy.special import logsumexp
from scipy import linalg

def _estimate_gaussian_parameters(X, resp, reg_covar):

    nk = resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps
    means = np.dot(resp.T, X) / nk[:, np.newaxis]

    n_components, n_features = means.shape
    covariances = np.empty((n_components, n_features, n_features))
    for k in range(n_components):
        diff = X - means[k]
        covariances[k] = np.dot(resp[:, k] * diff.T, diff) / nk[k]
        covariances[k].flat[:: n_features + 1] += reg_covar
    return nk, means, covariances

def _compute_precision_cholesky(covariances):
     estimate_precision_error_message = (
        "Fitting the mixture model failed because some components have "
        "ill-defined empirical covariance (for instance caused by singleton "
        "or collapsed samples). Try to decrease the number of components, "
        "or increase reg_covar."
     )
     n_components, n_features, _ = covariances.shape
     precisions_chol = np.empty((n_components, n_features, n_features))
     for k, covariance in enumerate(covariances):
          try:
               cov_chol = linalg.cholesky(covariance, lower=True)
          except linalg.LinAlgError:
               raise ValueError(estimate_precision_error_message)
          precisions_chol[k] = linalg.solve_triangular(
               cov_chol, np.eye(n_features), lower=True
          ).T
     return precisions_chol

def _compute_log_det_cholesky(matrix_chol, n_features):
     """Compute the log-det of the cholesky decomposition of matrices.

     Parameters
     ----------
     matrix_chol : array-like
          Cholesky decompositions of the matrices.
          'full' : shape of (n_components, n_features, n_features)
          'tied' : shape of (n_features, n_features)
          'diag' : shape of (n_components, n_features)
          'spherical' : shape of (n_components,)

     covariance_type : {'full', 'tied', 'diag', 'spherical'}

     n_features : int
          Number of features.

     Returns
     -------
     log_det_precision_chol : array-like of shape (n_components,)
          The determinant of the precision matrix for each component.
     """
     n_components, _, _ = matrix_chol.shape
     log_det_chol = np.sum(
          np.log(matrix_chol.reshape(n_components, -1)[:, :: n_features + 1]), 1
     )
     return log_det_chol

def _estimate_log_gaussian_prob(n_components, X, precisions_chol, means):
     
     # Log of determinant of covariance matrix
     n_samples = len(X)
     n_features = X.shape[1]
     log_det = _compute_log_det_cholesky(precisions_chol, n_features)

     # Log of probabilities
     log_prob = np.empty((n_samples, n_components))
     for k, (mu, prec_chol) in enumerate(zip(means, precisions_chol)):
          y = np.dot(X, prec_chol) - np.dot(mu, prec_chol)
          log_prob[:, k] = np.sum(np.square(y), axis=1)

     # log gaussian likelihood
     return -0.5 * (n_features * np.log(2 * np.pi) + log_prob) + log_det

def _estimate_log_weights(weights):
     return np.log(weights)

def _estimate_weighted_log_prob(n_components, X, precisions_chol, means, weights):
     return _estimate_log_gaussian_prob(n_components, X, precisions_chol, means) + _estimate_log_weights(weights)

def sk_estimate_log_prob_resp(n_components, X, precisions_chol, means, weights):
    weighted_log_prob = _estimate_weighted_log_prob(n_components=n_components, X=X, precisions_chol=precisions_chol, means=means, weights=weights)
    log_prob_norm = logsumexp(weighted_log_prob, axis=1)
    with np.errstate(under="ignore"):
        # ignore underflow
        log_resp = weighted_log_prob - log_prob_norm[:, np.newaxis]
    return log_prob_norm, log_resp


def sk_e_step(n_components, X, precisions_chol, means, weights):
     """E step.

     Parameters
     ----------
     X : array-like of shape (n_samples, n_features)

     Returns
     -------
     log_prob_norm : float
          Mean of the logarithms of the probabilities of each sample in X

     log_responsibility : array, shape (n_samples, n_components)
          Logarithm of the posterior probabilities (or responsibilities) of
          the point of each sample in X.
     """
     log_prob_norm, log_resp = sk_estimate_log_prob_resp(n_components, X, precisions_chol, means, weights)
     return np.mean(log_prob_norm), log_resp


def _m_step(X, log_resp):
     """M step.

     Parameters
     ----------
     X : array-like of shape (n_samples, n_features)

     log_resp : array-like of shape (n_samples, n_components)
          Logarithm of the posterior probabilities (or responsibilities) of
          the point of each sample in X.
     """
     weights_, means_, covariances_ = _estimate_gaussian_parameters(
          X, np.exp(log_resp), 1e-6
     )
     weights_ /= weights_.sum()
     precisions_cholesky_ = _compute_precision_cholesky(
          covariances_
     )

     return precisions_cholesky_, weights_, means_

In [93]:
np_prec_chol = gmm_init_params["precision_cholesky"].detach().numpy()
np_means = gmm_init_params["means"].detach().numpy()
np_weights = gmm_init_params["weights"].detach().numpy()

sk_log_gaussian_prob = _estimate_log_gaussian_prob(n_components=N_COMPONENTS, X=input_data, precisions_chol=np_prec_chol, means=np_means)
sk_weighted_long_prob = _estimate_weighted_log_prob(n_components=N_COMPONENTS, X=input_data, precisions_chol=np_prec_chol, means=np_means, weights=np_weights)
sk_log_prob_norm, sk_log_resp = sk_estimate_log_prob_resp(n_components=N_COMPONENTS, X=input_data, precisions_chol=np_prec_chol, means=np_means, weights=np_weights)
sk_e_step_a, sk_e_step_b = sk_e_step(n_components=N_COMPONENTS, X=input_data, precisions_chol=np_prec_chol, means=np_means, weights=np_weights)

In [94]:
sk_m_prec_chol, sk_m_weights, sk_m_means = _m_step(X=input_data, log_resp=sk_e_step_b)

In [86]:
torch_log_resp[0]

tensor([-5.6400e+01, -2.4667e+01, -1.0073e+01, -1.1522e+01, -2.4538e+01,
        -2.6805e+01, -2.9733e+01, -3.1089e+01, -7.4801e+01, -3.2550e+01,
        -2.1851e+01, -1.0096e+01, -2.3091e+01, -4.5575e+01, -5.2834e-04,
        -8.5151e+00, -1.9881e+01, -1.2752e+01, -3.4889e+01, -4.3621e+01,
        -5.7187e+01, -2.8923e+01, -5.3805e+01, -8.9127e+00, -9.2404e+00],
       grad_fn=<SelectBackward0>)

In [83]:
sk_e_step_b[0]

array([-5.63998944e+01, -2.46670077e+01, -1.00725121e+01, -1.15218768e+01,
       -2.45382049e+01, -2.68053977e+01, -2.97329862e+01, -3.10889246e+01,
       -7.48009445e+01, -3.25501940e+01, -2.18506808e+01, -1.00958383e+01,
       -2.30908069e+01, -4.55752849e+01, -5.28542998e-04, -8.51512977e+00,
       -1.98809137e+01, -1.27524528e+01, -3.48892326e+01, -4.36214176e+01,
       -5.71870157e+01, -2.89227726e+01, -5.38048764e+01, -8.91272089e+00,
       -9.24037549e+00])

In [87]:
torch_log_prob_norm

tensor(-1.0545, grad_fn=<MeanBackward0>)

In [81]:
sk_e_step_a

-1.05454014620348

In [98]:
print(sk_m_prec_chol[0][0])
print(torch_prec_chol[0][0])

[ 0.08028357  0.07119598  0.03184662  0.16216018  0.24411083  0.30974147
 -0.09739906  0.05201701 -0.80128211  0.69050011 -0.27069737 -0.23454647
 -0.64370607  0.50027748 -0.94783816  0.46651462  0.04549614 -0.40560012]
tensor([ 0.0803,  0.0712,  0.0318,  0.1622,  0.2441,  0.3097, -0.0974,  0.0520,
        -0.8013,  0.6905, -0.2706, -0.2346, -0.6437,  0.5003, -0.9479,  0.4665,
         0.0456, -0.4055], grad_fn=<SelectBackward0>)


In [101]:
print(sk_m_means[0])
print(torch_means[0])

[-32.12413403   7.2256603   26.57156942  17.81663587   0.34315073
  15.93977472   2.64068482  53.07602351  -9.55154226  42.8279865
 -35.28012696  34.8014331  -19.15105752  -7.43147729   6.31421727
  21.30646527   4.60899316   3.76961423]
tensor([-32.1241,   7.2257,  26.5716,  17.8166,   0.3432,  15.9398,   2.6407,
         53.0760,  -9.5515,  42.8280, -35.2801,  34.8014, -19.1511,  -7.4315,
          6.3142,  21.3065,   4.6090,   3.7696], grad_fn=<SelectBackward0>)


In [103]:
print(sk_m_weights)
print(torch_weights)

[0.00208807 0.02526938 0.06002368 0.05185892 0.02027081 0.01133898
 0.00893643 0.01342201 0.0013526  0.00896451 0.03813464 0.08378932
 0.04918676 0.00183637 0.13309347 0.08099787 0.03013934 0.12571496
 0.01406869 0.00294209 0.00125728 0.01173547 0.00128451 0.13813916
 0.08415472]
tensor([0.0021, 0.0253, 0.0600, 0.0519, 0.0203, 0.0113, 0.0089, 0.0134, 0.0014,
        0.0090, 0.0381, 0.0838, 0.0492, 0.0018, 0.1331, 0.0810, 0.0301, 0.1257,
        0.0141, 0.0029, 0.0013, 0.0117, 0.0013, 0.1381, 0.0842],
       grad_fn=<DivBackward0>)


# SK Learn Outputs

In [70]:
import numpy as np
from scipy import linalg

def sk_estimate_gaussian_parameters(X, resp, reg_covar):
    nk = (
        resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps
    )  # This adds small white noise to avoid division by zero
    means = np.dot(resp.T, X) / nk[:, np.newaxis]  # The centroids

    n_components, n_features = means.shape
    covariances = np.empty((n_components, n_features, n_features))
    for k in range(n_components):
        diff = X - means[k]
        covariances[k] = np.dot(resp[:, k] * diff.T, diff) / nk[k]
        covariances[k].flat[:: n_features + 1] += reg_covar

    return nk, means, covariances


def sk_compute_precision_cholesky(covariances: torch.Tensor):
    estimate_precision_error_message = (
        "Fitting the mixture model failed because some components have "
        "ill-defined empirical covariance (for instance caused by singleton "
        "or collapsed samples). Try to decrease the number of components, "
        "or increase reg_covar."
    )

    n_components, n_features, _ = covariances.shape
    precisions_chol = np.empty((n_components, n_features, n_features))
    for k, covariance in enumerate(covariances):
        try:
            cov_chol = linalg.cholesky(covariance, lower=True)
        except linalg.LinAlgError:
            raise ValueError(estimate_precision_error_message)
        precisions_chol[k] = linalg.solve_triangular(
            cov_chol, np.eye(n_features), lower=True
        ).T

    return precisions_chol

In [71]:
from sklearn.cluster import KMeans

n_samples, _ = input_data.shape
n_components = N_COMPONENTS

# K-means initialisation
resp = np.zeros((n_samples, n_components))
label = (
    KMeans(n_clusters=n_components, n_init=1, random_state=0)
    .fit(input_data)
    .labels_
)
resp[np.arange(n_samples), label] = 1
# Initialise GMM

n_samples, _ = input_data.shape
weights, means, covariances = None, None, None

weights, means, covariances = sk_estimate_gaussian_parameters(
    input_data, resp, 1e-6
)
weights /= n_samples

weights_ = weights
means_ = means
covariances_ = covariances

precisions_cholesky_ = sk_compute_precision_cholesky(covariances)

In [72]:
precisions_cholesky_[0][0]

array([ 0.09219085,  0.0889303 ,  0.02281819,  0.19152657,  0.29779096,
        0.30174624, -0.11401279,  0.0758457 , -0.80188263,  0.27832801,
       -0.77204619, -0.17212504, -0.50637951,  0.68939058, -0.71485848,
       -0.2383334 ,  0.46413801, -0.12564614])

In [73]:
weights * n_samples

array([  50.,  253., 1637.,  474.,  425., 3605.,  358.,   66., 1340.,
         23., 4134., 1708.,  373.,  568., 1958.,  165., 3618.,  189.,
         58.,   54.,  361.,   32.,   48.,  132., 3371.])

# SK Learn GMM 1 Epoch

In [144]:
from sklearn.mixture import GaussianMixture
skgmm = GaussianMixture(n_components=N_COMPONENTS, covariance_type='full', max_iter=1)
skgmm.fit(input_data)

/Users/shengchai/.local/share/virtualenvs/OpenSynth-EhRIPYd3/lib/python3.11/site-packages/sklearn/mixture/_base.py:270: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(


GaussianMixture(max_iter=1, n_components=25)

In [150]:
skgmm.weights_.shape

(25,)

In [157]:
a1 = skgmm._estimate_log_weights().shape

In [158]:
a2 = skgmm._estimate_log_prob(input_data).shape

In [156]:
skgmm._estimate_weighted_log_prob(input_data).shape

(25000, 25)

In [159]:
a1 + a2

(25, 25000, 25)